In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# ------------------ Simulation Functions ------------------

def conserved(rho, vx, vy, P, gamma, vol):
    Mass = rho * vol
    Momx = rho * vx * vol
    Momy = rho * vy * vol
    Energy = (P/(gamma-1) + 0.5*rho*(vx**2+vy**2)) * vol
    return Mass, Momx, Momy, Energy

def primitives(Mass, Momx, Momy, Energy, gamma, vol):
    rho = Mass / vol
    vx  = Momx / (rho * vol)
    vy  = Momy / (rho * vol)
    P   = (Energy/vol - 0.5*rho*(vx**2+vy**2)) * (gamma-1)
    rho, vx, vy, P = setGhostCells(rho, vx, vy, P)
    return rho, vx, vy, P

def grad(f, dx):
    R = -1  # right shift
    L = 1   # left shift
    f_dx = (np.roll(f, R, axis=0) - np.roll(f, L, axis=0)) / (2*dx)
    f_dy = (np.roll(f, R, axis=1) - np.roll(f, L, axis=1)) / (2*dx)
    f_dx, f_dy = setGhostGradients(f_dx, f_dy)
    return f_dx, f_dy

def slope(f, dx, f_dx, f_dy):
    R = -1
    L = 1
    f_dx = np.maximum(0., np.minimum(1., ((f - np.roll(f, L, axis=0))/dx) / (f_dx + 1.0e-8*(f_dx==0)))) * f_dx
    f_dx = np.maximum(0., np.minimum(1., (-(f - np.roll(f, R, axis=0))/dx) / (f_dx + 1.0e-8*(f_dx==0)))) * f_dx
    f_dy = np.maximum(0., np.minimum(1., ((f - np.roll(f, L, axis=1))/dx) / (f_dy + 1.0e-8*(f_dy==0)))) * f_dy
    f_dy = np.maximum(0., np.minimum(1., (-(f - np.roll(f, R, axis=1))/dx) / (f_dy + 1.0e-8*(f_dy==0)))) * f_dy
    return f_dx, f_dy

def extrapolateInSpaceToFace(f, f_dx, f_dy, dx):
    R = -1
    f_XL = f - f_dx * dx/2
    f_XL = np.roll(f_XL, R, axis=0)
    f_XR = f + f_dx * dx/2
    f_YL = f - f_dy * dx/2
    f_YL = np.roll(f_YL, R, axis=1)
    f_YR = f + f_dy * dx/2
    return f_XL, f_XR, f_YL, f_YR

def inFlux(F, flux_F_X, flux_F_Y, dx, dt):
    R = -1
    L = 1
    F += - dt * dx * flux_F_X
    F +=   dt * dx * np.roll(flux_F_X, L, axis=0)
    F += - dt * dx * flux_F_Y
    F +=   dt * dx * np.roll(flux_F_Y, L, axis=1)
    return F

def outFlux(rho_L, rho_R, vx_L, vx_R, vy_L, vy_R, P_L, P_R, gamma):
    en_L = P_L/(gamma-1) + 0.5*rho_L*(vx_L**2 + vy_L**2)
    en_R = P_R/(gamma-1) + 0.5*rho_R*(vx_R**2 + vy_R**2)
    rho_star  = 0.5*(rho_L + rho_R)
    momx_star = 0.5*(rho_L*vx_L + rho_R*vx_R)
    momy_star = 0.5*(rho_L*vy_L + rho_R*vy_R)
    en_star   = 0.5*(en_L + en_R)
    P_star    = (gamma-1)*(en_star - 0.5*(momx_star**2 + momy_star**2) / rho_star)
    
    flux_Mass   = momx_star
    flux_Momx   = momx_star**2/rho_star + P_star
    flux_Momy   = momx_star * momy_star/rho_star
    flux_Energy = (en_star + P_star) * momx_star/rho_star
    
    C_L = np.sqrt(gamma*P_L/rho_L) + np.abs(vx_L)
    C_R = np.sqrt(gamma*P_R/rho_R) + np.abs(vx_R)
    rho_star  = 0.5*(rho_L + rho_R)
    momx_star = 0.5*(rho_L*vx_L + rho_R*vx_R)
    C   = np.maximum(C_L, C_R)
    
    flux_Mass   -= C * 0.5 * (rho_L - rho_R)
    flux_Momx   -= C * 0.5 * (rho_L*vx_L - rho_R*vx_R)
    flux_Momy   -= C * 0.5 * (rho_L*vy_L - rho_R*vy_R)
    flux_Energy -= C * 0.5 * (en_L - en_R)
    
    return flux_Mass, flux_Momx, flux_Momy, flux_Energy

def addGhostCells(rho, vx, vy, P):
    rho = np.hstack((rho[:,0:1], rho, rho[:,-1:]))
    vx  = np.hstack((vx[:,0:1], vx, vx[:,-1:]))
    vy  = np.hstack((vy[:,0:1], vy, vy[:,-1:]))
    P   = np.hstack((P[:,0:1], P, P[:,-1:]))
    return rho, vx, vy, P

def setGhostCells(rho, vx, vy, P):
    rho[:,0]  = rho[:,1]
    vx[:,0]   = vx[:,1]
    vy[:,0]   = -vy[:,1]
    P[:,0]    = P[:,1]
    
    rho[:,-1] = rho[:,-2]
    vx[:,-1]  = vx[:,-2]
    vy[:,-1]  = -vy[:,-2]
    P[:,-1]   = P[:,-2]
    
    return rho, vx, vy, P

def setGhostGradients(f_dx, f_dy):
    f_dy[:,0]  = -f_dy[:,1]
    f_dy[:,-1] = -f_dy[:,-2]
    return f_dx, f_dy

def SourceTerms(Mass, Momx, Momy, Energy, g, dt):
    Energy += dt * Momy * g
    Momy   += dt * Mass * g
    return Mass, Momx, Momy, Energy

# ------------------ Simulation with FuncAnimation ------------------

def main():
    # Simulation parameters
    N = 64                    # resolution N x 3N
    boxsizeX = 0.5
    boxsizeY = 1.5
    gamma_val = 1.4           # ideal gas gamma
    courant_fac = 0.4
    tEnd = 2.5
    tOut = 0.1                # output frequency for animation
    useSlopeLimiting = False
    
    # Mesh
    dx = boxsizeX / N
    vol = dx**2
    xlin = np.linspace(0.5*dx, boxsizeX-0.5*dx, N)
    ylin = np.linspace(0.5*dx, boxsizeY-0.5*dx, 3*N)
    Y, X = np.meshgrid(ylin, xlin)
    
    # Initial conditions: heavy fluid on top of light, with a perturbation
    g = -9.81       # gravity
    w0 = 0.00025
    P0 = 25
    rho = 1.0 + (Y > 0.75)
    vx = np.zeros_like(X)
    vy = w0 * (1 - np.cos(4*np.pi*X)) * (1 - np.cos(4*np.pi*Y/3))
    P  = P0 + g*(Y - 0.75)*rho
    rho, vx, vy, P = addGhostCells(rho, vx, vy, P)
    
    # Initial conserved variables
    Mass, Momx, Momy, Energy = conserved(rho, vx, vy, P, gamma_val, vol)
    
    # Store simulation state in a dictionary
    sim = {
        "t": 0.0,
        "Mass": Mass,
        "Momx": Momx,
        "Momy": Momy,
        "Energy": Energy,
        "dx": dx,
        "vol": vol,
        "gamma": gamma_val,
        "courant_fac": courant_fac,
        "g": g,
        "useSlopeLimiting": useSlopeLimiting
    }
    
    # # Set up figure and initial image
    fig, ax = plt.subplots(figsize=(16,12), dpi=500)
    # Get initial primitive variables (for plotting)
    rho, vx, vy, P = primitives(sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"], sim["gamma"], sim["vol"])
    im = ax.imshow(rho.T, animated=True, origin='upper')
    im.set_clim(0.8, 2.2)
    ax.invert_yaxis()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_aspect('equal')
    
    total_frames = int(tEnd / tOut) + 1

    def update(frame):
        # Advance simulation until reaching the next output time
        target_time = frame * tOut
        while sim["t"] < target_time and sim["t"] < tEnd:
            # Compute primitive variables from conserved state
            rho, vx, vy, P = primitives(sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"], sim["gamma"], sim["vol"])
            # Compute adaptive time step (CFL condition)
            dt_candidate = sim["courant_fac"] * np.min(sim["dx"] / (np.sqrt(sim["gamma"] * P/rho) + np.sqrt(vx**2 + vy**2)))
            dt = dt_candidate
            # Do not overshoot the target time
            if sim["t"] + dt > target_time:
                dt = target_time - sim["t"]
            # Add gravitational source term (half-step)
            sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"] = SourceTerms(sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"], sim["g"], dt/2)
            # Update primitive variables after source term
            rho, vx, vy, P = primitives(sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"], sim["gamma"], sim["vol"])
            # Compute gradients
            rho_dx, rho_dy = grad(rho, sim["dx"])
            vx_dx,  vx_dy  = grad(vx, sim["dx"])
            vy_dx,  vy_dy  = grad(vy, sim["dx"])
            P_dx,   P_dy   = grad(P, sim["dx"])
            # Apply slope limiting if enabled
            if sim["useSlopeLimiting"]:
                rho_dx, rho_dy = slope(rho, sim["dx"], rho_dx, rho_dy)
                vx_dx, vx_dy   = slope(vx, sim["dx"], vx_dx, vx_dy)
                vy_dx, vy_dy   = slope(vy, sim["dx"], vy_dx, vy_dy)
                P_dx, P_dy     = slope(P, sim["dx"], P_dx, P_dy)
            # Extrapolate half-step in time
            rho_prime = rho - 0.5*dt*(vx*rho_dx + rho*vx_dx + vy*rho_dy + rho*vy_dy)
            vx_prime  = vx  - 0.5*dt*(vx*vx_dx + vy*vx_dy + (1/rho)*P_dx)
            vy_prime  = vy  - 0.5*dt*(vx*vy_dx + vy*vy_dy + (1/rho)*P_dy)
            P_prime   = P   - 0.5*dt*(sim["gamma"]*P*(vx_dx+vy_dy) + vx*P_dx + vy*P_dy)
            # Extrapolate in space to face centers
            rho_XL, rho_XR, rho_YL, rho_YR = extrapolateInSpaceToFace(rho_prime, rho_dx, rho_dy, sim["dx"])
            vx_XL, vx_XR, vx_YL, vx_YR = extrapolateInSpaceToFace(vx_prime, vx_dx, vx_dy, sim["dx"])
            vy_XL, vy_XR, vy_YL, vy_YR = extrapolateInSpaceToFace(vy_prime, vy_dx, vy_dy, sim["dx"])
            P_XL, P_XR, P_YL, P_YR = extrapolateInSpaceToFace(P_prime, P_dx, P_dy, sim["dx"])
            # Compute fluxes (local Lax-Friedrichs/Rusanov)
            flux_Mass_X, flux_Momx_X, flux_Momy_X, flux_Energy_X = outFlux(rho_XL, rho_XR, vx_XL, vx_XR, vy_XL, vy_XR, P_XL, P_XR, sim["gamma"])
            flux_Mass_Y, flux_Momy_Y, flux_Momx_Y, flux_Energy_Y = outFlux(rho_YL, rho_YR, vy_YL, vy_YR, vx_YL, vx_YR, P_YL, P_YR, sim["gamma"])
            # Update conserved variables using the computed fluxes
            sim["Mass"]   = inFlux(sim["Mass"], flux_Mass_X, flux_Mass_Y, sim["dx"], dt)
            sim["Momx"]   = inFlux(sim["Momx"], flux_Momx_X, flux_Momx_Y, sim["dx"], dt)
            sim["Momy"]   = inFlux(sim["Momy"], flux_Momy_X, flux_Momy_Y, sim["dx"], dt)
            sim["Energy"] = inFlux(sim["Energy"], flux_Energy_X, flux_Energy_Y, sim["dx"], dt)
            # Add gravitational source term (half-step)
            sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"] = SourceTerms(sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"], sim["g"], dt/2)
            # Increment simulation time
            sim["t"] += dt

        # After advancing to the target time, update the image data
        rho, vx, vy, P = primitives(sim["Mass"], sim["Momx"], sim["Momy"], sim["Energy"], sim["gamma"], sim["vol"])
        im.set_array(rho.T)
        return [im]

    ani = animation.FuncAnimation(fig, update, frames=total_frames, interval=50, blit=True, repeat=False)
    
    # Saving animation
    Writer = animation.FFMpegWriter(fps=60, metadata=dict(artist='Yugesh'), bitrate=800)
    ani.save('finitevolume2_animation_2.mp4', writer=Writer, dpi=240)
    plt.close(fig)
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# -------------------------
# Simulation Parameters
# -------------------------
Lx = 1.0           # Domain size in x
Lz = 1.0           # Domain size in z
Nx = 128           # Number of grid points in x
Nz = 128           # Number of grid points in z
dx = Lx / Nx
dz = Lz / Nz

# Physical parameters
nu = 1.0e-3        # Kinematic viscosity
kappa = 1.0e-3     # Diffusivity for density (used for numerical stability)
g = 9.81           # Gravity acceleration (acts downward)
# Define densities: heavy fluid (upper) and light fluid (lower)
rho_top = 1.0      # Heavy fluid density (upper region)
rho_bot = 0.9      # Light fluid density (lower region)

# Time stepping
dt = 1e-3
max_steps = 2000       # Total simulation steps
steps_per_frame = 10   # Simulation steps per animation frame

# -------------------------
# Grid Setup
# -------------------------
# Create 1D arrays and a 2D meshgrid with indexing='ij'
x = np.linspace(0, Lx, Nx, endpoint=False)
z = np.linspace(0, Lz, Nz, endpoint=False)
X, Z = np.meshgrid(x, z, indexing='ij')  # X and Z have shape (Nx, Nz)

# -------------------------
# Field Arrays
# -------------------------
# Velocity components: u (x-direction) and w (z-direction)
u = np.zeros((Nx, Nz))
w = np.zeros((Nx, Nz))

# Pressure field
p = np.zeros((Nx, Nz))

# Density field (acting as our scalar, now representing fluid density)
rho = np.zeros((Nx, Nz))

# -------------------------
# Initial Conditions: Density Field
# -------------------------
# Heavy fluid on top (rho_top) and light fluid at the bottom (rho_bot)
mid_z = Lz / 2.0
rho[:, :] = rho_bot  # Set entire domain initially to light fluid density

# Define a 1D interface with a sinusoidal perturbation (function of x)
pert_amp = 0.01
interface = mid_z + pert_amp * np.sin(2.0 * np.pi * x / Lx)

# Loop over grid points: assign rho_top above the perturbed interface and rho_bot below
for i in range(Nx):
    for j in range(Nz):
        if Z[i, j] > interface[i]:
            rho[i, j] = rho_top
        else:
            rho[i, j] = rho_bot

# -------------------------
# Helper Functions
# -------------------------
def laplacian(field):
    """Compute the 2D Laplacian using central differences."""
    d2fdx2 = (np.roll(field, -1, axis=0) - 2*field + np.roll(field, +1, axis=0)) / (dx*dx)
    d2fdz2 = (np.roll(field, -1, axis=1) - 2*field + np.roll(field, +1, axis=1)) / (dz*dz)
    return d2fdx2 + d2fdz2

def advect(field, u_field, w_field, dt):
    """Advect a scalar field using centered differences."""
    fxp = np.roll(field, -1, axis=0)
    fxm = np.roll(field, +1, axis=0)
    fzp = np.roll(field, -1, axis=1)
    fzm = np.roll(field, +1, axis=1)
    dfdx = (fxp - fxm) / (2*dx)
    dfdz = (fzp - fzm) / (2*dz)
    return field - dt * (u_field * dfdx + w_field * dfdz)

def compute_divergence(u_field, w_field):
    """Compute the divergence of the velocity field."""
    du_dx = (np.roll(u_field, -1, axis=0) - np.roll(u_field, +1, axis=0)) / (2*dx)
    dw_dz = (np.roll(w_field, -1, axis=1) - np.roll(w_field, +1, axis=1)) / (2*dz)
    return du_dx + dw_dz

def pressure_poisson(u_star, w_star):
    """
    Solve the pressure Poisson equation:
      ∇²p = (1/dt)*∇·(u_star, w_star)
    using an iterative Jacobi method.
    """
    global p
    div_u = compute_divergence(u_star, w_star) / dt
    beta = (dx*dx*dz*dz) / (2*(dx*dx + dz*dz))
    for _ in range(50):
        p_old = p.copy()
        p_xp = np.roll(p, -1, axis=0)
        p_xm = np.roll(p, +1, axis=0)
        p_zp = np.roll(p, -1, axis=1)
        p_zm = np.roll(p, +1, axis=1)
        p = ((p_xp + p_xm)*dz*dz + (p_zp + p_zm)*dx*dx - div_u*dx*dx*dz*dz) * beta
        # Enforce simple BCs on pressure at the top and bottom
        p[:, 0] = p[:, 1]
        p[:, -1] = p[:, -2]
        if np.max(np.abs(p - p_old)) < 1e-6:
            break

def projection_step(u_star, w_star):
    """
    Project the intermediate velocity field onto divergence-free fields using
    the computed pressure gradient.
    """
    p_xp = np.roll(p, -1, axis=0)
    p_xm = np.roll(p, +1, axis=0)
    dpdx = (p_xp - p_xm) / (2*dx)
    p_zp = np.roll(p, -1, axis=1)
    p_zm = np.roll(p, +1, axis=1)
    dpdz = (p_zp - p_zm) / (2*dz)
    u_new = u_star - dt * dpdx
    w_new = w_star - dt * dpdz
    # Impose no-slip conditions at the top and bottom boundaries
    u_new[:, 0] = 0.0
    u_new[:, -1] = 0.0
    w_new[:, 0] = 0.0
    w_new[:, -1] = 0.0
    return u_new, w_new

def simulation_step():
    global u, w, rho
    # 1) Advect velocity fields
    u_star = advect(u, u, w, dt)
    w_star = advect(w, u, w, dt)
    # 2) Viscous diffusion of velocity
    u_star += nu * dt * laplacian(u)
    w_star += nu * dt * laplacian(w)
    # 3) Buoyancy force: heavier fluid should sink.
    # Use a reference density (here the average of top and bottom)
    rho0 = (rho_top + rho_bot) / 2.0
    # Add a buoyancy term so that if density > rho0, acceleration is downward
    w_star += - dt * g * (rho - rho0) / rho0
    # 4) Enforce incompressibility via pressure projection
    pressure_poisson(u_star, w_star)
    u_new, w_new = projection_step(u_star, w_star)
    # 5) Advect and diffuse the density field
    rho_star = advect(rho, u_new, w_new, dt)
    rho_star += kappa * dt * laplacian(rho)
    # Impose fixed density at the boundaries (Dirichlet BCs)
    rho_star[:, 0] = rho_bot
    rho_star[:, -1] = rho_top
    # Update fields
    u[:] = u_new
    w[:] = w_new
    rho[:] = rho_star

# -------------------------
# Set Up Animation
# -------------------------
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(rho.T, origin='lower', extent=[0, Lx, 0, Lz],
               aspect='auto', cmap='viridis', vmin=rho_bot, vmax=rho_top)
fig.colorbar(im, label='Density')
ax.set_title("Rayleigh–Taylor Instability (Density Field)")
ax.set_xlabel("x")
ax.set_ylabel("z")

def init():
    im.set_data(rho.T)
    return [im]

def update(frame):
    for _ in range(steps_per_frame):
        simulation_step()
    im.set_data(rho.T)
    ax.set_title(f"Step {frame*steps_per_frame}")
    return [im]

ani = animation.FuncAnimation(fig, update, frames=max_steps // steps_per_frame,
                              init_func=init, blit=False, interval=50)

# Save the animation
ani.save('rayleigh_taylor_instability_1.mp4', writer='ffmpeg', fps=30, dpi=240)
# plt.show()

# plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Densities
rho_min, rho_max = 0.1, 2.0
npoints = 400
rho1 = np.linspace(rho_min, rho_max, npoints)
rho2 = np.linspace(rho_min, rho_max, npoints)

# Grid of densities
RHO1, RHO2 = np.meshgrid(rho1, rho2)

# Classify the regions:
# Unstable for rho1 > rho2 (Atwood number A > 0)
# Stable for rho1 < rho2 (Atwood number A < 0)
region = np.zeros_like(RHO1)
region[RHO1 > RHO2] = 1   # Mark unstable region as 1
region[RHO1 < RHO2] = -1  # Mark stable region as -1

# Contour plot
plt.figure(figsize=(8, 6))

contours = plt.contourf(RHO1, RHO2, region, levels=[-1.1, -0.1, 0.1, 1.1], colors=['blue', 'red'])
plt.xlabel(r'$\rho_1$ (Density of top fluid)')
plt.ylabel(r'$\rho_2$ (Density of bottom fluid)')
plt.title('Stability Regions Based on \n$\rho_1$ vs $\rho_2$ (Rayleigh-Taylor Instability)')

plt.plot([rho_min, rho_max], [rho_min, rho_max], 'k--', linewidth=2, label=r'Neutral: $\rho_1=\rho_2$')

# Adding a legend
plt.legend(loc='upper left')

# Optionally, add a colorbar without numeric ticks since we are using categorical information
cbar = plt.colorbar(contours, ticks=[-1, 0, 1])
cbar.ax.set_yticklabels(['Stable', 'Neutral', 'Unstable'])
cbar.set_label('Stability Region')

plt.tight_layout()
plt.show()


In [ ]:
try:
    import numpy as np
    import matplotlib.pyplot as plt
except ModuleNotFoundError as e:
    print("One of the required modules is not installed. Please install numpy and matplotlib to run this script.")
    raise e
# Densities
rho_min, rho_max = 0.1, 2.0
npoints = 400
rho1 = np.linspace(rho_min, rho_max, npoints)
rho2 = np.linspace(rho_min, rho_max, npoints)

# Grid of densities
RHO1, RHO2 = np.meshgrid(rho1, rho2)

# Classify the regions:
# Unstable for rho1 > rho2 (dense fluid on top of lighter fluid)
# Stable for rho1 < rho2 (lighter fluid above denser fluid)
region = np.zeros_like(RHO1)
region[RHO1 > RHO2] = 1   # Unstable region
region[RHO1 < RHO2] = -1  # Stable region

plt.figure(figsize=(8, 6))

contours = plt.contourf(RHO1, RHO2, region, levels=[-1.1, 0, 1.1], colors=['green', 'red'])
plt.xlabel(r'$\rho_1$ (Density of top fluid)')
plt.ylabel(r'$\rho_2$ (Density of bottom fluid)')

plt.title(r'Stability Regions Based on $\rho_1$ vs $\rho_2$ (Rayleigh-Taylor Instability)')

# plt.plot([rho_min, rho_max], [rho_min, rho_max], 'k--', linewidth=2, label=r'Neutral: $\rho_1=\rho_2$')

# plt.legend(loc='upper left')

cbar = plt.colorbar(contours, ticks=[-1, 1])
cbar.ax.set_yticklabels(['Stable', 'Unstable'])
cbar.set_label('Stability Region')

plt.tight_layout()
plt.savefig('stability_regions.pdf', dpi=800)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Toy model parameters
#   We assume a functional form:
#       γ²(k*², λ*) = M(λ*) · x · exp(-b · x)
#
#   with x = k*² and
#
#       M(λ*) = 1 - c · (λ* + 0.5)²,
#
# so that the maximum amplitude (i.e. maximum instability) occurs at λ* = -0.5.
#
# Here, c and b are constants that adjust the amplitude and the x-scale,
# respectively.
# -----------------------------------------------------------------------------
c = 0.2   # controls reduction of growth away from λ* = -0.5
b = 1.0   # sets the exponential decay rate

def gamma_squared(x, lam):
    M = 1 - c * (lam + 0.5)**2
    M = max(M, 0)  # ensure nonnegative amplitude factor
    return M * x * np.exp(-b * x)

# Define an array for x = k*²
x_vals = np.linspace(0, 10, 400)

# Choose several values for λ* to compare; note that λ* = -0.5 gives the maximum instability.
lambda_values = [-0.5, 1.0, 1.5]

# Create the plot
plt.figure(figsize=(8, 6))

for lam in lambda_values:
    gamma2_vals = np.array([gamma_squared(x, lam) for x in x_vals])
    plt.plot(x_vals, gamma2_vals, label=f"λ* = {lam}")

plt.xlabel("Normalized squared wave number $k^{*2}$")
plt.ylabel("Normalized squared growth rate $γ^2$")
plt.title("Toy Model: Normalized Growth Rate vs. Normalized Wave Number")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Dimensionless parameters
omega_fx = 0.5
omega_fz = 0.5
h_star   = 1.0
n        = 1.0
z_star   = 1.0
g_star   = 10.0
kx2_ratio = 0.3   # k*_x^2 = 0.2 * k*^2

# Normalized wavenumber range
k_star = np.linspace(0.1, 10, 800)
k2     = k_star**2
kx2    = kx2_ratio * k2

# λ* values to plot
lambda_vals = [-0.5, 1.0, 1.5]

# Container for γ^2 curves
gamma2_vals = {}

# Precompute constant
pi_n_over_h = n * np.pi / h_star

for lam in lambda_vals:
    lam2 = lam**2

    # Build each term of Eq. (24)
    A1 = lam**4 + pi_n_over_h**4 - 6*lam2*pi_n_over_h**2
    A2 = 2*lam**3 - 6*lam*pi_n_over_h**2
    A3 = (5/4 - k2)*(lam2 - pi_n_over_h**2)
    A4 = lam*(1/4 - k2)
    Term1 = (A1 + A2 + A3 + A4) * omega_fz**2

    B1 = 2*lam*(lam2 - pi_n_over_h**2)
    B2 = 3*(lam2 - pi_n_over_h**2)
    B3 = lam*(5/4 - 2*k2)
    B4 = (1/4 - k2)
    Term2 = B1 + B2 + B3 + B4

    C1 = 2*lam + 1
    C2 = 2*lam2 + 2*lam - 2*pi_n_over_h**2 + (1/4 - k2)
    Term3 = C1*kx2*omega_fx**2 - C1*C2*omega_fz**2

    DenomFactor = 6*lam2 + 6*lam - 2*pi_n_over_h**2 + (5/4 - 2*k2)

    TermNum2 = Term2 * Term3 / DenomFactor
    Term4    = (-kx2*omega_fx**2*(lam2 - pi_n_over_h**2)
               - lam*kx2*omega_fx**2
               + k2*kx2*omega_fx**2
               - 3*k2*g_star*z_star**2*np.exp(-z_star))
    Numerator   = Term1 + TermNum2 + Term4

    D1 = Term2
    D2 = 2*lam*z_star**3*np.exp(-z_star) + 3*z_star**2*np.exp(-z_star)
    Den1 = -D1 * D2 / DenomFactor
    Den2 = (z_star**3*np.exp(-z_star)*(lam2 - pi_n_over_h**2)
           + 3*lam*z_star**2*np.exp(-z_star)
           - k2*z_star**3*np.exp(-z_star))
    Denominator = Den1 + Den2

    # Final γ^2
    gamma2_vals[lam] = Numerator / Denominator - 1

# Plot
plt.figure(figsize=(8,5))
for lam in lambda_vals:
    plt.plot(k2, gamma2_vals[lam], label=f'λ* = {lam}')
plt.xlabel(r'$k^{*2}$')
plt.ylabel(r'$\gamma^2$')
plt.xlim(0, 100)
plt.ylim(0, 10)
plt.title('Square Normalized Growth Rate vs. Squared Wave-Number')
plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig('growth_rate_vs_wave_number.pdf', dpi=300)
plt.show()


In [ ]:
# Plot
plt.figure(figsize=(8, 5))
for lam in lambda_vals:
    plt.plot(k2, gamma2_vals[lam], label=f'λ* = {lam}, ω_fx = ω_fz = {omega_fz}')
plt.xlabel(r'$k^{*2}$')
plt.ylabel(r'$\gamma^2$')
plt.xlim(0, 100)
plt.ylim(0, 10)
plt.title('Square Normalized Growth Rate vs. Squared Wave-Number')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('growth_rate_vs_wave_number_with_omega.pdf', dpi=300)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define density ranges
rho1 = np.linspace(0.1, 10, 100)
rho2 = np.linspace(0.1, 10, 100)
R1, R2 = np.meshgrid(rho1, rho2)

# Compute Atwood number and threshold omega*
At = (R2 - R1) / (R2 + R1)
omega_th = np.sqrt(np.clip(At, 0, None))  # Real only where At >= 0

# Plot 3D surface
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(R1, R2, omega_th, edgecolor='black')

# Label axes
ax.set_xlabel('Density ρ₁')
ax.set_ylabel('Density ρ₂')
ax.set_zlabel('Threshold ω*')
ax.set_title('Threshold Dimensionless Magnetic-Field Frequency ω* vs ρ₁ and ρ₂')

plt.tight_layout()
plt.savefig('threshold_omega_vs_density.pdf', dpi=300)
plt.show()


In [ ]:
import plotly.graph_objects as go

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(z=omega_th, x=R1, y=R2, colorscale='viridis')])

# Update layout for better visualization
fig.update_layout(
    title="Threshold Dimensionless Magnetic-Field Frequency ω* vs ρ₁ and ρ₂",
    scene=dict(
        xaxis_title="Density ρ₁",
        yaxis_title="Density ρ₂",
        zaxis_title="Threshold ω*"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the interactive plot
fig.show()

In [ ]:
fig.write_html(
  "threshold_surface.html",
  include_plotlyjs="cdn",      # small file, relies on online JS
  full_html=False,             # exports only the <div> with its script
  default_width="100%",        # so it fills its container
  default_height="100%"
)

In [ ]:
# at the end of your script, before fig.show():
fig.write_image("threshold_surface.png", width=1200, height=800, scale=2)
# or for vector:
fig.write_image("threshold_surface.pdf")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Parameters
rho1 = 2.0  # Density of heavier fluid
rho2 = 1.0  # Density of lighter fluid
A = (rho1 - rho2) / (rho1 + rho2)  # Atwood number
g = 9.81  # Gravitational acceleration (m/s^2)
# mu0 = 4 * np.pi * 1e-7  # Permeability of free space (H/m)
mu0 = 1.0  # Parameterized Magnetic permeability (H/m)
# Wave number range
k = np.linspace(0.1, 10, 500)

# Different magnetic field strengths (assuming k · B = k * B for simplicity i.e. \theta = 0)
B_values = [0, 0.5, 0.75, 1.0, 1.25, 1.5]
# B_values = [1.25, 1.5]



# 2D line plots of γ^2 vs k for each B
plt.figure(figsize=(10, 6))
for B in B_values:
    gamma2 = A * g * k - (k * B)**2 / (mu0 * (rho1 + rho2))
    plt.plot(k, gamma2, label=f'B = {B}')
#     crossing_indices = np.where(np.diff(np.sign(gamma2)))[0]

# # Extract the corresponding k values for these crossings
#     k_c = k[crossing_indices]
#     k_mid = k_c / 2
#     for kc in k_c:
#         plt.scatter(kc, 0, color='red')  # Mark the crossing points
#         plt.text(kc, 0.5, f'$k_c={kc:.2f}$', color='red', fontsize=10, ha='center')
#     plt.scatter(k_mid, 0, color='green', label=f'Mid-point $k_{{mid}}={k_mid[0]:.2f}$')
#     plt.text(k_mid, 0.5, f'$k_{{mid}}={k_mid[0]:.2f}$', color='green', fontsize=10, ha='center')
plt.axhline(0, linestyle='--')
plt.xlabel('Wave number k')
plt.ylabel(r'Growth Rate ($\gamma^2$)')
plt.title('Growth rate squared vs k for different magnetic field strengths')
plt.legend()
# plt.xlim(0, 7)
# plt.ylim(0, 6)
plt.xlim(0, 10)
plt.ylim(0, 30)
plt.grid(True)
# plt.savefig('growth_rate_for_two_B.pdf', dpi=500)
plt.tight_layout()
plt.savefig('growth_rate_vs_k_for_B.pdf', dpi=500)
plt.show()

# 3D surface plot of γ^2 as a function of k and B
# B_range = np.linspace(0, 1.0, 100)
# K, B_mesh = np.meshgrid(k, B_range)
# Gamma2_surface = A * g * K - (K * B_mesh)**2 / (mu0 * (rho1 + rho2))

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(K, B_mesh, Gamma2_surface)
# ax.set_xlabel('Wave number k')
# ax.set_ylabel('Magnetic field B')
# ax.set_zlabel(r'$\gamma^2$')
# ax.set_title('Growth rate squared surface vs k and B')

# plt.show()


In [ ]:
# Calculate the mid-point of the range from k=0 to k=kc
k_mid = k_c / 2

# Plot the graph with annotations
plt.figure(figsize=(10, 6))
plt.plot(k, gamma2, label=r'$\gamma^2$ vs $k$', color='blue')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8, label='x-axis')

# Annotate the crossing points
for kc in k_c:
    plt.scatter(kc, 0, color='red')  # Mark the crossing points
    plt.text(kc, 0.5, f'$k_c={kc:.2f}$', color='red', fontsize=10, ha='center')

# Annotate the mid-point
plt.scatter(k_mid, 0, color='green', label=f'Mid-point $k_{{mid}}={k_mid[0]:.2f}$')
plt.text(k_mid, 0.5, f'$k_{{mid}}={k_mid[0]:.2f}$', color='green', fontsize=10, ha='center')

# Add labels, title, and legend
plt.xlabel('Wave number $k$')
plt.ylabel(r'$\gamma^2$ (Growth Rate Squared)')
plt.title('Growth Rate Squared vs Wave Number with Critical and Mid Points Annotated')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the graph with k_mid annotated
plt.figure(figsize=(10, 6))
plt.plot(k, gamma2, label=r'$\gamma^2$ vs $k$', color='blue')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8, label='x-axis')

# Annotate the crossing points
for kc in k_c:
    plt.scatter(kc, 0, color='red')  # Mark the crossing points
    plt.text(kc, 0.5, f'$k_c={kc:.2f}$', color='red', fontsize=10, ha='center')

# Annotate the mid-point
plt.scatter(k_mid, 0, color='green', label=f'Mid-point $k_{{mid}}={k_mid[0]:.2f}$')
plt.text(k_mid, 0.5, f'$k_{{mid}}={k_mid[0]:.2f}$', color='green', fontsize=10, ha='center')

# Add labels, title, and legend
plt.xlabel('Wave number $k$')
plt.ylabel(r'$\gamma^2$ (Growth Rate Squared)')
plt.title('Growth Rate Squared vs Wave Number with Critical and Mid Points Annotated')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
crossing_indices = np.where(np.diff(np.sign(gamma2)))[0]

# Extract the corresponding k values for these crossings
k_c = k[crossing_indices]

# Plot the graph with annotations
plt.figure(figsize=(10, 6))
plt.plot(k, gamma2, label=r'$\gamma^2$ vs $k$', color='blue')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8, label='x-axis')

# Annotate the crossing points
for kc in k_c:
    plt.scatter(kc, 0, color='red')  # Mark the crossing points
    plt.text(kc, 0.5, f'$k_c={kc:.2f}$', color='red', fontsize=10, ha='center')

# Add labels, title, and legend
plt.xlabel('Wave number $k$')
plt.ylabel(r'$\gamma^2$ (Growth Rate Squared)')
plt.title('Growth Rate Squared vs Wave Number with Critical Points Annotated')
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
import plotly.graph_objects as go

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(z=Gamma2_surface, x=K, y=B_mesh, colorscale='Viridis')])

# Update layout for better visualization
fig.update_layout(
    title="Magnetic Field vs Wave Number vs Growth Rate Squared",
    scene=dict(
        xaxis_title="Wave Number (k)",
        yaxis_title="Magnetic Field (B)",
        zaxis_title="Growth Rate Squared (γ²)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the interactive plot
fig.show()

In [ ]:
fig.write_html(
  "mag_field.html",
  include_plotlyjs="cdn",      # small file, relies on online JS
  full_html=False,             # exports only the <div> with its script
  default_width="100%",        # so it fills its container
  default_height="100%"
)

In [ ]:
import plotly.graph_objects as go

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(z=Gamma2_surface, x=K, y=B_mesh, colorscale='Viridis')])

# Update layout for better visualization
fig.update_layout(
    title="Magnetic Field vs Wave Number vs Growth Rate Squared",
    scene=dict(
        xaxis_title="Wave Number (k)",
        yaxis_title="Magnetic Field (B)",
        zaxis_title="Growth Rate Squared (γ²)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the interactive plot
fig.show()

In [ ]:
# Fixed values of k to plot
k_values = [0.5, 1.0, 2.0, 5.0, 10.0]  # Example values of k

# Create the plot
plt.figure(figsize=(8, 6))

for k_fixed in k_values:
    gamma2_B = A * g * k_fixed - (k_fixed * B_range)**2 / (mu0 * (rho1 + rho2))
    plt.plot(B_range, gamma2_B, label=f'k = {k_fixed}')

plt.axhline(0, linestyle='--', color='black', linewidth=0.8)
plt.xlabel('Magnetic Field (B)')
plt.ylabel(r'$\gamma^2$ (Growth Rate Squared)')
plt.title('Magnetic Field vs Growth Rate Squared for Fixed k')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create a meshgrid for rho1 and rho2
rho1_range = np.linspace(0.1, 10, 100)
rho2_range = np.linspace(0.1, 10, 100)
RHO1, RHO2 = np.meshgrid(rho1_range, rho2_range)

# Compute Atwood number
At = (RHO2 - RHO1) / (RHO2 + RHO1)

# Compute magnetic field B (assuming a relationship with Atwood number)
B = np.sqrt(np.clip(At, 0, None))  # Real values only where At >= 0

# Plot the 3D surface
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(RHO1, RHO2, B, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel(r'$\rho_1$ (Density of top fluid)')
ax.set_ylabel(r'$\rho_2$ (Density of bottom fluid)')
ax.set_zlabel('Magnetic Field $B$')
ax.set_title('Magnetic Field $B$ vs $\\rho_1$ vs $\\rho_2$')

# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)

plt.tight_layout()
plt.show()

In [ ]:
# Fixed values of k to plot
k_values = [0.5, 1.0, 2.0, 5.0]  # Example values of k

# Create the plot
plt.figure(figsize=(8, 6))

for k_fixed in k_values:
    gamma2_B = A * g * k_fixed - (k_fixed * B_range)**2 / (mu0 * (rho1 + rho2))
    plt.plot(B_range, gamma2_B, label=f'k = {k_fixed}')

plt.axhline(0, linestyle='--', color='black', linewidth=0.8)
plt.xlabel('Magnetic Field (B)')
plt.ylabel(r'$\gamma^2$ (Growth Rate Squared)')
plt.title('Magnetic Field vs Growth Rate Squared for Fixed k')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(B_range, Gamma2_surface, label=f'B = {B}')

In [ ]:
plt.plot(rho1, omega_th[:, 0], label='ρ₂ = 0.1')
plt.plot(rho1, rho2)
plt.plot(rho2, omega_th[:, 0], label='ρ₂ = 10')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
rho2 = 2.0
rho1 = 1.0
A_t = (rho2 - rho1) / (rho2 + rho1)
g = 9.81
mu0 = 4 * np.pi * 1e-7
mu0 = 1.0  # Parameterized Magnetic permeability (H/m)
B0 = 1.0

# Wave number range and height range
k = np.linspace(0.1, 10, 500)
z_values = [-1, -0.5, -0.25, 0.0, 0.25, 0.5, 1.0]  # Different heights

# 2D plots
plt.figure(figsize=(10, 6))
for z in z_values:
    gamma2 = A_t * g * k - (B0**2 * np.exp(2 * z) / (mu0 * (rho1 + rho2))) * k**2
    plt.plot(k, gamma2, label=f'z = {z}')
plt.axhline(0, linestyle='--')
plt.xlabel('Wave number k')
plt.ylabel('γ²')
plt.title('γ² vs k for different z')
plt.legend()
plt.grid(True)
# plt.xlim(0, 4)
plt.ylim(0, 10)
plt.savefig('gamma_vs_k_z.pdf', dpi=500)
plt.tight_layout()
plt.show()

# # 3D plot
# from mpl_toolkits.mplot3d import Axes3D  # this import can be here
# z = np.linspace(0, 1, 100)
# K, Z = np.meshgrid(k, z)
# Gamma2 = A_t * g * K - (B0**2 * np.exp(2 * Z) / (mu0 * (rho1 + rho2))) * K**2

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(K, Z, Gamma2)
# ax.set_xlabel('k')
# ax.set_ylabel('z')
# ax.set_zlabel('γ²')
# ax.set_title('γ² surface vs k and z')
# plt.show()
